In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import h5py
import pandas as pd
import os
import sys
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import math
import pdb
import scipy
# Add project path to path for import
project_path = os.path.abspath("..")
if project_path not in sys.path:
    sys.path.append(project_path)

# Add module path to path for import
module_path = os.path.abspath("../data_utility/data_utility.py")
if module_path not in sys.path:
    sys.path.append(module_path)
    
from data_utility import CREAM_Day # class to work with a day of the CREAM Dataset
import glob

In [10]:
PATH_TO_DATA = os.path.abspath(os.path.join("..", "..", "Datasets", "CREAM"))
ALL_DAYS = ["2018-08-23" , "2018-08-24" , "2018-08-25",  "2018-08-26" , "2018-08-27" , "2018-08-28" ,
"2018-08-29", "2018-08-30", "2018-08-31", "2018-09-01", "2018-09-02" , "2018-09-03" ,  "2018-09-04" ,
"2018-09-05", "2018-09-06", "2018-09-07", "2018-09-08" , "2018-09-09" , "2018-09-10", "2018-09-11", "2018-09-12" 
"2018-09-13" ,"2018-09-14" ,"2018-09-15" ,  "2018-09-16", "2018-09-17", "2018-09-18","2018-09-19"  , "2018-09-20" ,
"2018-09-21" , "2018-09-22" ,  "2018-09-23" ,"2018-09-24" ,"2018-09-25" ,"2018-09-26" , "2018-09-27", "2018-09-28" ,
"2018-09-29" , "2018-09-30" , "2018-10-01" ,"2018-10-02" , "2018-10-03" ,"2018-10-04", "2018-10-05" , "2018-10-06" ,
"2018-10-07", "2018-10-08" ]

In [11]:
# Load the events  using an arbirtrary dummy day
day_path = os.path.join(PATH_TO_DATA, "2018-08-24") #arbitrary day to initialize the object
current_CREAM_day = CREAM_Day(cream_day_location=day_path,use_buffer=True, buffer_size_files=2) 

# Load the electrical component events (the raw ones)
all_component_events = current_CREAM_day.load_component_events(os.path.join(PATH_TO_DATA, "component_events.csv"), filter_day=False)


In [12]:
all_component_events.head()

,Filename,Timestamp,Amplitude,Event_Type,ID,Component,Date
0,coffee-maker-2018-08-23T06-00-03.676161T+0200-...,2018-08-23 06:01:15.637567+02:00,-1,On,0,unlabeled,2018-08-23
1,coffee-maker-2018-08-23T06-00-03.676161T+0200-...,2018-08-23 06:01:17.500380+02:00,6,Off,1,unlabeled,2018-08-23
2,coffee-maker-2018-08-23T06-00-03.676161T+0200-...,2018-08-23 06:01:36.927880+02:00,-1,On,2,unlabeled,2018-08-23
3,coffee-maker-2018-08-23T06-00-03.676161T+0200-...,2018-08-23 06:01:38.790692+02:00,6,Off,3,unlabeled,2018-08-23
4,coffee-maker-2018-08-23T06-00-03.676161T+0200-...,2018-08-23 06:02:28.798817+02:00,0,On,4,unlabeled,2018-08-23


## Plot the start-up shape (current signal) of the three labeked component types

In [ ]:

for j in np.arange(start=20, stop=50):
    # Get one event from each component. 

    fig, axes = plt.subplots(nrows=1, ncols=6)
    duration = 1 #seconds

    for i, component_name in enumerate(["heater", "millingplant", "pump"]):
    
        component_event = all_component_events[all_component_events.Component == component_name].iloc[j]

        # Initialize the respective CREAM_Day object for the event of interest
        day_path = os.path.join(PATH_TO_DATA, str(component_event.Date))
        current_CREAM_day = CREAM_Day(cream_day_location=day_path, use_buffer=True, buffer_size_files=2) 

        if component_name is "pump": #to get the right portion of the signal for the pump
            start_timestamp = component_event.Timestamp - timedelta(seconds=0.1)
        else:
            start_timestamp = component_event.Timestamp + timedelta(seconds=0.3) #to load some time before the event happens
        # Use the CREAM_Day object to load the timeframe of interest, using the convenience function provided by the class
        # The functions expects the start timestamp of the event and the duration of the window to load (i.e. the window size to load)
        org_voltage, org_current = current_CREAM_day.load_time_frame(start_datetime=start_timestamp, duration=duration) #and WINDOW_SIZE seconds after the event

        # Downsample the current to faciliate the plotting
        current = org_current[::10]
        axes[i].plot(np.arange(len(current)), current)
        axes[i].title.set_text(component_name)
        axes[i].set_ylabel("Ampere")
        axes[i].set_xlabel("Seconds")
        axes[i].set_xticks(np.arange(0,duration+0.1,len(current)))
        axes[i].set_xticklabels([0,0.25,0.5])
        axes[i].set_ylim(-20,20)
    
        # also plot the current rms to show the shape of the signal
        # We have a sampling rate of 6400 samples per second and a mains frequency of 50 Hz
        # Per cycle, we have 6400 / 50 = 128 samples
        samples_per_period = int(6400 / 100) #samples for two periods means 50*2=100
        org_current_chunks = np.array_split(org_current, samples_per_period)
        rms_values = []
        for c in org_current_chunks:
            rms_values.append(np.sqrt(np.mean(np.square(c))))
                              
        axes[i+3].plot(np.arange(len(rms_values)), rms_values)
        axes[i+3].title.set_text(component_name + "current rms")
        axes[i+3].set_ylabel("Ampere RMS")
        axes[i+3].set_xlabel("Seconds")
        axes[i+3].set_xticks(np.arange(0,duration+0.1,len(rms_values)))
        axes[i+3].set_xticklabels([0,0.25,0.5])
        #axes[i+3].set_ylim(-20,20)
        
    fig.tight_layout()
    plt.show()
      
    
    

In [22]:
# or just use an espressor for an example
all_product_events = current_CREAM_day.load_machine_events(os.path.join(PATH_TO_DATA, "product_events.csv"), raw_file=False, filter_day=False)


In [23]:
all_product_events[all_product_events.Event_Type == "espresso"]

,Start_Timestamp,Automatic_Timestamp,Event_Type,End_Timestamp,Event_Duration_Seconds,Date
0,2018-08-23 07:31:02.583281+02:00,2018-08-23 07:32:00+02:00,latte_macchiato,2018-08-23 07:31:35.405781+02:00,54.822500,2018-08-23
1,2018-08-23 07:32:13.051094+02:00,2018-08-23 07:32:00+02:00,hot_water,2018-08-23 07:32:34.873594+02:00,21.822500,2018-08-23
2,2018-08-23 08:33:44.309063+02:00,2018-08-23 08:34:00+02:00,coffee,2018-08-23 08:34:12.518750+02:00,28.209687,2018-08-23
3,2018-08-23 08:35:14.115625+02:00,2018-08-23 08:35:00+02:00,cappuccino,2018-08-23 08:35:59.357500+02:00,45.241875,2018-08-23
4,2018-08-23 08:36:10.922031+02:00,2018-08-23 08:36:00+02:00,cappuccino,2018-08-23 08:36:58.825156+02:00,47.903125,2018-08-23
